# Instruction Fidelity Testing Exercise

### Objective
In this exercise, you'll evaluate how well language models follow specific instructions using Hugging Face's transformers library. By analyzing the fidelity of model responses to given prompts, you'll identify improvement areas in both model performance and instruction formulation.

### Step 0: Install Required Libraries
First, ensure you have the necessary libraries installed.


In [ ]:
!pip install transformers
import pandas as pd
from transformers import pipeline

### Step 1: Load a Pre-trained Language Model
Load a Hugging Face model for text generation. Here, we'll use GPT-2, but you may experiment with others like EleutherAI/gpt-neo-1.3B.

In [ ]:
# Load the model
model_name = "gpt2"
generator = pipeline("text-generation", model=model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


### Step 2: Prepare New Prompts with Specific Instructions
Use the prompts below, which differ from the original demo. Modify or add your prompts for variety.

In [ ]:
# Prompts for testing
prompts = [
    "Summarize in two sentences: The sun is the primary energy source for life on Earth.",
    "List three benefits of reading daily.",
    "Write a concise paragraph on the advantages of solar panels.",
    "Explain why exercise is essential in one sentence.",
]

### Step 3: Generate Responses
Pass the prompts to the model and capture the results.

In [ ]:
# Generate responses
results = []
for prompt in prompts:
    output = generator(prompt, max_length=150, num_return_sequences=1)
    results.append({"Prompt": prompt, "Generated Response": output[0]["generated_text"]})

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### Step 4: Organize Results into a Table
Format the results for analysis using a pandas DataFrame.

In [ ]:
# Create a DataFrame
df = pd.DataFrame(results)
df["Generated Response"] = df.apply(
    lambda row: row["Generated Response"].replace(row["Prompt"], "").strip(), axis=1
)
df

,Prompt,Generated Response
0,Summarize in two sentences: The sun is the pri...,"It is what powers plants, fish and animals, wh..."
1,List three benefits of reading daily.,"First, if you want to read more quickly, you c..."
2,Write a concise paragraph on the advantages of...,"""One of the key things to know about making so..."
3,Explain why exercise is essential in one sente...,"As we'll see below, every athlete has an exerc..."


### Step 5: Evaluate Adherence to Instructions
Analyze how well the responses align with the instructions. Fill in the function below to check adherence for each prompt.

In [ ]:
# Analyze adherence
def analyze_adherence(prompt, response):
    response = response.strip()
    if "Summarize in two sentences" in prompt:
        return response.count(".") == 2
    elif "List three benefits" in prompt:
        return len(response.split("\n")) >= 3
    elif "Write a concise paragraph" in prompt:
        return len(response.split()) > 30
    elif "Explain why exercise is essential" in prompt:
        return response.count(".") == 1
    else:
        return "Unknown instruction type"

# Apply adherence check
df["Adherence"] = df.apply(lambda row: analyze_adherence(row["Prompt"], row["Generated Response"]), axis=1)
df

,Prompt,Generated Response,Adherence
0,Summarize in two sentences: The sun is the pri...,"It is what powers plants, fish and animals, wh...",False
1,List three benefits of reading daily.,"First, if you want to read more quickly, you c...",True
2,Write a concise paragraph on the advantages of...,"""One of the key things to know about making so...",True
3,Explain why exercise is essential in one sente...,"As we'll see below, every athlete has an exerc...",False


### Step 6: Analyze and Report Findings
Identify failure cases and brainstorm improvements.

In [ ]:
# Display the results
print("Instruction Fidelity Analysis")
print(df)

# Highlight failures
failure_cases = df[df["Adherence"] == False]
print("\nFailure Cases:")
print(failure_cases)

Instruction Fidelity Analysis
                                              Prompt  \
0  Summarize in two sentences: The sun is the pri...   
1              List three benefits of reading daily.   
2  Write a concise paragraph on the advantages of...   
3  Explain why exercise is essential in one sente...   

                                  Generated Response  Adherence  
0  It is what powers plants, fish and animals, wh...      False  
1  First, if you want to read more quickly, you c...       True  
2  "One of the key things to know about making so...       True  
3  As we'll see below, every athlete has an exerc...      False  

Failure Cases:
                                              Prompt  \
0  Summarize in two sentences: The sun is the pri...   
3  Explain why exercise is essential in one sente...   

                                  Generated Response  Adherence  
0  It is what powers plants, fish and animals, wh...      False  
3  As we'll see below, every athlete has a

### Questions for Testing Model Performance:
1. Does the model's failure indicate a misunderstanding of the instruction's context (e.g., "two sentences" misunderstood as "brief summary") or an inherent limitation in its ability to process and execute the task correctly?
2. How accurately does the model adhere to the specific constraints provided in the instructions, such as sentence limits, item counts, or word counts?
3. Does the model focus more on being creative or detailed instead of strictly following the instructions?